In [1]:
pip install asyncpraw


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd


In [3]:
review_keywords = (
    r"(good service|bad service|friendly staff|rude crew|helpful|unhelpful|"
    r"flew|fly|flown|took a flight|onboard|aircraft|seating|legroom|"
    r"on time|delayed|canceled|missed flight|"
    r"comfortable|cramped|smooth flight|bumpy|turbulence|"
    r"cheap ticket|expensive|affordable|overpriced|"
    r"baggage|lost luggage|extra charge|carry-on|"
    r"good experience|bad experience|would recommend|never again|worst airline)"
)
unrelated_keywords = (
    r"(Thai Airways|other airline|different airline|another airline|better airline|\?$"
    r"|but people here aren't really shy about checking stuff|Cathay Pacific|IATA|hand carry rule|"
    r"OP|That’s a great itinerary|scoot|Fernandes|Maayos|Thanks|pyramid scheme)"
)

In [4]:
import asyncpraw

reddit = asyncpraw.Reddit(
    client_id="",
    client_secret="",
    user_agent="",
    username="",
    password=""
    )
    

In [5]:
import re
import asyncio
import asyncpraw

async def getAirAsiaReviews(url, unrelatedKeywords):
    

    try:
        submission = await reddit.submission(url=url)
        await submission.comments.replace_more(limit=None)  # Load all comments

        airasia_reviews = []
        total_comments = 0  # Counter for all comments & subcomments

        async def process_comment(comment):
            """ Recursively process a comment and its replies """
            nonlocal total_comments
            total_comments += 1  # Count this comment

            comment_text = comment.body.strip()

            # Check comment conditions
            if len(comment_text.split()) > 1 and not any(re.search(keyword, comment_text, re.IGNORECASE) for keyword in unrelatedKeywords):
                airasia_reviews.append(comment_text)

            # Fetch and process replies
            await comment.replies.replace_more(limit=None)  # Ensure all replies are loaded
            for reply in comment.replies:
                await process_comment(reply)  # Recursively process each reply

        # Process all top-level comments
        for top_comment in submission.comments:
            await process_comment(top_comment)

        return airasia_reviews, total_comments  # Return reviews & total count

    except asyncpraw.exceptions.APIException as e:
        print(f"API Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e.__class__.__name__}: {e}")
    return [], 0

In [6]:
async def processUrls(urls_with_keywords):
    all_reviews = []
    total_comments_count = 0  # Initialize counter

    # Create tasks with corresponding keywords for each URL
    tasks = [getAirAsiaReviews(url, keywords) for url, keywords in urls_with_keywords]

    results = await asyncio.gather(*tasks)  # Get results

    for reviews, count in results:
        all_reviews.extend(reviews)
        total_comments_count += count  # Add count from each URL

    print(f"Total Comments (including subcomments): {total_comments_count}")  
    print(f"Filtered Comments: {len(all_reviews)}")  

    for review in all_reviews:
        print("-" * 20)
        print(review)

In [75]:
urls_with_keywords = [
   ("https://www.reddit.com/r/malaysia/comments/11qssb9/airasia_now_offers_10_million_free_seats_for/", ["literally", "lmao", "hilarious", "sun", "cancellation", "oxymoron", "often", "mcmc", "kite", "headaches"])
    ]

In [76]:
await processUrls(urls_with_keywords)

Total Comments (including subcomments): 22
Filtered Comments: 11
--------------------
I wouldn’t fly air Asia even if they paid me to do so. The subsequent headache that comes with it just isn’t worth it.
--------------------
And canceled it last minute, makes you buy new ticket since you already paid for the vacation accommodation, attractions, transportation.
--------------------
I don't even need to read the article to know there's a catch. It's either not free or subject to extremely inconvenient terms.

All I know is, if I was in a room with a cobra, a crocodile and AirAsia and I had a gun with 2 bullets, I'd shoot AirAsia twice.

Edit: Yea it's not free. It's one of those free seats\* but you pay all the taxes. Those seats were going to be empty anyway, so they give those out, you help cover the taxes, and then they will try to get you with excess baggage fees, food and merch sales onboard.
--------------------
Do not trust AirAsia, countless vacations has been ruined because of 